In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK：在线解释自动机器学习训练表格分类模型

<table align="left"> 
  <td style="text-align: center"> 
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_automl_tabular_classification_online_explain.ipynb"> 
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> 在Colab中打开 
    </a> 
  </td> 
  <td style="text-align: center"> 
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fexplainable_ai%2Fsdk_automl_tabular_classification_online_explain.ipynb"> 
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> 在Colab Enterprise中打开 
    </a> 
  </td> 
  <td style="text-align: center"> 
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/explainable_ai/sdk_automl_tabular_classification_online_explain.ipynb"> 
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> 在工作台中打开 
    </a> 
  </td> 
  <td style="text-align: center"> 
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/explainable_ai/sdk_automl_tabular_classification_online_explain.ipynb"> 
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> 在GitHub上查看 
    </a> 
  </td> 
</table>

## 概述

本教程演示了如何使用Vertex AI SDK创建表格式分类模型，并使用Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行在线预测及解释。

了解更多有关[表格数据分类](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)和[Vertex可解释AI](https://cloud.google.com/vertex-ai/docs/explainable-ai/overview)的信息。

### 目标

在本教程中，您将学习如何使用 AutoML 从Python脚本创建一个表格二元分类模型。然后，您将学习如何使用 Vertex AI 预测服务来进行带解释的在线预测请求。您也可以选择使用 `gcloud` 命令行工具或在 Cloud Console 上在线创建和部署模型。

本教程使用以下谷歌云 ML 服务：

- Vertex AI AutoML
- Vertex AI 预测
- Vertex 可解释AI
- Vertex AI 模型资源
- Vertex AI 端点资源

执行的步骤包括：

- 创建一个 Vertex AI 数据集资源。
- 训练一个 AutoML 表格二元分类模型。
- 查看训练模型的模型评估指标。
- 创建一个服务端点资源。
- 部署模型资源到服务端点资源。
- 进行带解释的在线预测请求。
- 取消部署模型资源。

数据集

本教程使用的数据集是来自[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/overview)的[Iris数据集](https://www.tensorflow.org/datasets/catalog/iris)。该数据集不需要任何特征工程。本教程中使用的数据集版本存储在公共Cloud Storage桶中。训练好的模型可以预测三种不同Iris花的品种：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

### 费用

本教程使用 Google Cloud 的计费组件：

- Vertex AI
- Cloud Storage

了解[Vertex AI 价格](https://cloud.google.com/vertex-ai/pricing)和[Cloud Storage 价格](https://cloud.google.com/storage/pricing)，并使用[Pricing 计算器](https://cloud.google.com/products/calculator/)根据您的预期使用量生成费用估算。

开始吧!

### 为Python安装Vertex AI SDK和其他必需的软件包

### 安装Vertex AI SDK for Python和其他必需的软件包

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 google-cloud-storage 
! pip3 install --upgrade --quiet tensorflow  
! pip3 install -U --quiet tabulate  

### 重新启动运行时（仅适用于Colab）

为了使用新安装的包，您必须在Google Colab上重新启动运行时。

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 内核将要重新启动。在继续到下一步之前，请等待它完成。⚠️</b>
</div>

### 在谷歌 Colab 上验证您的笔记本环境

在谷歌 Colab 上验证您的环境。

In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### 设置 Google Cloud 项目信息

要开始使用 Vertex AI，您必须拥有一个现有的 Google Cloud 项目。了解如何[设置项目和开发环境](https://cloud.google.com/vertex-ai/docs/start/cloud-environment)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶不存在时才运行以下单元格创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### 初始化 Python 环境下的 Vertex AI SDK

为您的项目和对应的存储桶初始化 Python 环境下的 Vertex AI SDK。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在您已经准备好创建自己的AutoML表格分类模型。

将云存储培训数据的位置设置为`IMPORT_FILE`变量。

In [ ]:
IMPORT_FILE = "gs://cloud-samples-data/tables/iris_1000.csv"

快速查看您的数据

您可以通过数CSV索引文件中的行数 (`wc -l`) 来计算示例的数量，然后查看前几行。

标签列的标题名称是`label_column`，它是CSV文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

### 创建数据集

接下来，使用`TabularDataset`类的`create`方法创建数据集，该方法接受以下参数：

- `display_name`：数据集的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入数据集。
- `bq_source`：另外，您可以将数据项从BigQuery表导入数据集。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.TabularDataset.create(
    display_name="Iris", gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：
1）创建一个训练管道。
2）指定您的训练参数并运行作业。

#### 创建训练管道

使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：训练作业的可读名称。
- `optimization_prediction_type`：模型要产生的预测类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name="iris",
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(job)

#### 运行训练流程

通过调用`run`方法来启动训练作业，使用以下参数：

- `dataset`：用于训练模型的数据集。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集的百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集的百分比。
- `validation_fraction_split`：用于验证的数据集的百分比。
- `target_column`：要训练为标签的列名。
- `budget_milli_node_hours`：（可选）以千小时为单位指定的最大训练时间。
- `disable_early_stopping`： 如果为`True`，服务可能认为无法进一步改进模型目标测量，训练可能会在使用完整预算之前完成。

完成`run`方法后，将返回模型资源。

训练流程的执行可能需要最多20分钟。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="iris",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

## 回顾模型评估分数

在模型训练完成后，您可以使用`list_model_evaluations()`方法来查看评估分数。该方法会为每个评估切片返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

部署模型

接下来，将您的模型部署到在线预测中。要部署模型，请调用`deploy`方法，使用以下参数：

- `machine_type`：计算机类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送一个带有可解释性的在线预测请求，向部署的模型发送。在这种方法中，预测的响应包括一个解释，说明特征是如何对解释有所贡献的。

创建测试项目

为了演示目的，使用合成数据生成预测。

In [ ]:
INSTANCE = {
    "petal_length": "1.4",
    "petal_width": "1.3",
    "sepal_length": "5.1",
    "sepal_width": "2.8",
}

### 通过解释生成预测

现在您的模型已部署到一个端点，您可以通过向端点发送预测请求来获得在线解释。

#### 请求

每个实例的格式为：

    [特征列表]

由于`explain()`方法可以接受多个项目（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

从`explain()`调用的响应是一个包含以下条目的Python字典：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类别标签的类别名称。
- `confidences`：对于分类，每个类别标签的预测置信度值，介于0和1之间。
- `values`：对于回归，预测的值。
- `deployed_model_id`：生成预测的Vertex AI部署模型资源的标识符。
- `explanations`：Explainable AI返回的特征归因。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.explain(instances_list)
print(prediction)

### 理解解释回应

看一下模型预测并将其与实际值进行比较。

In [ ]:
import numpy as np

try:
    label = np.argmax(prediction[0][0]["scores"])
    cls = prediction[0][0]["classes"][label]
    print("Predicted Value:", cls, prediction[0][0]["scores"][label])
except:
    pass

### 检查特征归因

接下来看看这个特定示例的特征归因。正面归因值意味着特定特征将您的模型预测值上升了该量，负面归因值则相反。

In [ ]:
from tabulate import tabulate

feature_names = ["sepal_length", "sepal_width", "petal_length", "petal_width"]
attributions = prediction.explanations[0].attributions[0].feature_attributions

rows = []
for i, val in enumerate(feature_names):
    rows.append([val, INSTANCE[val], attributions[val]])
print(tabulate(rows, headers=["Feature name", "Feature value", "Attribution value"]))

检查您的解释和基线

为了更好地理解您得到的特征归因，您应该将它们与您模型的基线进行比较。在大多数情况下，您的归因值加上基线应该非常接近于每个输入的模型预测值。还要注意，对于回归模型，解释返回的`baseline_score`对于发送到您模型的每个示例都是相同的。对于分类模型，每个类别都有自己的基线。

在本节中，发送10个测试示例给您的模型进行预测，以便将特征归因与基线进行比较。然后，通过`sanity_check_explanations` 方法运行每个测试示例的归因。

In [ ]:
import random

# Prepare 10 test examples to your model for prediction using a random distribution to generate
# test instances
instances = []
for i in range(10):
    pl = str(random.uniform(1.0, 2.0))
    pw = str(random.uniform(1.0, 2.0))
    sl = str(random.uniform(4.0, 6.0))
    sw = str(random.uniform(2.0, 4.0))
    instances.append(
        {"petal_length": pl, "petal_width": pw, "sepal_length": sl, "sepal_width": sw}
    )

response = endpoint.explain(instances)

#### 健全性检查

在下面的函数中，您对解释进行了健全性检查。

In [ ]:
import numpy as np


def sanity_check_explanations(
    explanation, prediction, mean_tgt_value=None, variance_tgt_value=None
):
    passed_test = 0
    total_test = 1
    # `attributions` is a dict where keys are the feature names
    # and values are the feature attributions for each feature
    baseline_score = explanation.attributions[0].baseline_output_value
    print("baseline:", baseline_score)

    # Sanity check 1
    # The prediction at the input is equal to that at the baseline.
    #  Please use a different baseline. Some suggestions are: random input, training
    #  set mean.
    if abs(prediction - baseline_score) <= 0.05:
        print("Warning: example score and baseline score are too close.")
        print("You might not get attributions.")
    else:
        passed_test += 1
        print("Sanity Check 1: Passed")

    print(passed_test, " out of ", total_test, " sanity checks passed.")


i = 0
for explanation in response.explanations:
    try:
        prediction = np.max(response.predictions[i]["scores"])
    except TypeError:
        prediction = np.max(response.predictions[i])
    sanity_check_explanations(explanation, prediction)
    i += 1

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
# Undeploy the model from the endpoint
endpoint.undeploy_all()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

# Delete the dataset
dataset.delete()

# Delete the automl training job
job.delete()

# Delete the Cloud Storage bucket
delete_bucket = False  # set True for deletion
if delete_bucket:
    ! gsutil rm -r $BUCKET_URI